In [1]:
import pandas as pd
import numpy as np
import wbgapi as wb
import requests
import string
import gzip, re, os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.relpath(
    "brave-aviary-319513-f0d0183ea603.json"
)
client = bigquery.Client()

In [3]:
its = pd.read_csv(
    "https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/bop_its6_det.tsv.gz",
    sep="\t",
    compression="gzip",
    index_col=False,
)

In [4]:
its_timespan_range = range(2010, 2020)
its_timespan = list(its_timespan_range)
its_timespan_quoted = ["{0}".format(i) for i in its_timespan]
its_timespan_space_quoted = ["{0} ".format(i) for i in its_timespan]
its.rename(
    columns={k: v for k, v in zip(its_timespan_space_quoted, its_timespan)},
    inplace=True,
)

In [5]:
str_vars = ["currency", "bop_item", "stk_flow", "partner_iso_2", "geo_iso_2"]
str_vars_red = ["partner_iso_2", "geo_iso_2"]
its[str_vars] = its.iloc[:, 0].str.split(",", expand=True)
its.drop(its.columns[0], axis=1, inplace=True)
its = its[its["partner_iso_2"] != its["geo_iso_2"]].reset_index(drop=True)

In [6]:
its[its_timespan] = its[its_timespan].apply(pd.to_numeric, errors="coerce")
its[str_vars] = its[str_vars].astype(str)

In [7]:
eu28_iso_2 = [
    "AT",
    "BE",
    "BG",
    "CY",
    "CZ",
    "DK",
    "DE",
    "EE",
    "ES",
    "FI",
    "FR",
    "GB",
    "GR",
    "HR",
    "HU",
    "IE",
    "IT",
    "LT",
    "LU",
    "LV",
    "MT",
    "NL",
    "PL",
    "PT",
    "RO",
    "SE",
    "SI",
    "SK",
]

In [8]:
str_vars = str_vars.remove("currency")
its.drop("currency", axis=1, inplace=True)

In [9]:
c_iso_2 = its.partner_iso_2.str.len() == 2
its = its[c_iso_2].dropna(subset=its_timespan, how="all").reset_index(drop=True)
its = its.replace("UK", "GB").replace("EL", "GR")
its = its[~(its.partner_iso_2 == "FZ")]

In [10]:
its_eu28 = its[its.geo_iso_2.isin(eu28_iso_2)].reset_index(drop=True)
its_eu28 = its_eu28[its_timespan + str_vars_red + ["bop_item", "stk_flow"]]

In [11]:
its_eu28_ip = pd.melt(
    its_eu28[(its_eu28.bop_item == "SJ112") & (its_eu28.stk_flow == "CRE")],
    id_vars=str_vars_red,
    value_vars=its_timespan,
    var_name="year",
    value_name="ip_export",
)

In [12]:
its_eu28_ip_sum = (
    its_eu28_ip.groupby(["partner_iso_2", "geo_iso_2"]).sum("ip_export").reset_index()
)
its_eu28_ip_sum = its_eu28_ip_sum[its_eu28_ip_sum.ip_export > 0]

In [13]:
its_eu28_ip_sum_within = its_eu28_ip_sum[
    its_eu28_ip_sum.partner_iso_2.isin(eu28_iso_2)
].reset_index(drop=True)
its_eu28_ip_sum_within[["geo_iso_2", "partner_iso_2", "ip_export"]].to_csv(
    "its_eu28_ip_sum_within.csv", index=False
)

In [14]:
its_eu28_ip_sum_outside = its_eu28_ip_sum[
    ~its_eu28_ip_sum.partner_iso_2.isin(eu28_iso_2)
].reset_index(drop=True)
its_eu28_ip_sum_outside[["geo_iso_2", "partner_iso_2", "ip_export"]].to_csv(
    "its_eu28_ip_sum_outside.csv", index=False
)

In [15]:
its_eu28_ip["iso_2_pair"] = (
    its_eu28_ip["geo_iso_2"] + "-" + its_eu28_ip["partner_iso_2"]
)

In [16]:
countries = list(set(list(its_eu28_ip.partner_iso_2) + list(its_eu28_ip.geo_iso_2)))
partners = list(its_eu28_ip.partner_iso_2.unique())

In [17]:
cc_iso = pd.read_csv(
    "https://gist.github.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv",
    usecols=["Country", "Alpha-2 code", "Alpha-3 code"],
    index_col=False,
)
cc_iso.rename(
    columns={"Country": "country", "Alpha-2 code": "iso_2", "Alpha-3 code": "iso_3"},
    inplace=True,
)
cc_iso = cc_iso.replace(' "', "", regex=True).replace('"', "", regex=True)
cc_iso_3_2_map = pd.Series(cc_iso.iso_2.values, index=cc_iso.iso_3).to_dict()
cc_country_iso_2_map = pd.Series(cc_iso.iso_2.values, index=cc_iso.country).to_dict()

In [18]:
tax = pd.read_csv(
    "https://github.com/TaxFoundation/worldwide-corporate-tax-rates/raw/master/final-data/final_data_long.csv",
    usecols=["iso_3", "year", "rate"],
    index_col=False,
)
tax = (
    tax
    .pivot(index="iso_3", columns="year", values="rate")
    .reset_index(level=0)
    .reset_index(level=0, drop=True)
    .rename_axis(None, axis=1)
)
tax["iso_2"] = tax.iso_3.map(cc_iso_3_2_map)
tax = (
    tax[tax.iso_2.isin(countries)][["iso_2"] + its_timespan]
    .reset_index(drop=True)
    .sort_values(by=["iso_2"], ascending=True)
)
tax_long = pd.melt(
    tax,
    id_vars="iso_2",
    value_vars=its_timespan,
    var_name="year",
    value_name="tax",
)
tax_eu28 = tax[tax.iso_2.isin(eu28_iso_2)].reset_index(drop=True)

In [19]:
tax_eu28 = tax[tax.iso_2.isin(eu28_iso_2)].reset_index(drop=True)
tax_eu28_long = tax_long[tax_long.iso_2.isin(eu28_iso_2)].reset_index(drop=True)
tax_eu28_long.to_csv("tax_eu28_long.csv", index=False)

In [20]:
stricter_control_thresholds_fixed = {"DE": 25}
stricter_control_thresholds_share = [
    "EE",
    "ES",
    "FI",
    "GB",
    "IT",
    "LV",
    "PL",
    "PT",
    "SE",
]
control_threshold_share = 0.8

In [21]:
gdp = wb.data.DataFrame("NY.GDP.MKTP.CD", time=its_timespan_range)
gdp = gdp.T.fillna(
    gdp.mean(axis=1)
).T
gdp = gdp.rename(columns=dict(zip(gdp.columns, its_timespan))).reset_index()
gdp = pd.melt(
    gdp,
    id_vars=["economy"],
    value_vars=its_timespan,
    var_name="year",
    value_name="gdp",
)
gdp["iso_2"] = gdp.economy.map(cc_iso_3_2_map)
gdp = gdp[(gdp.iso_2.isin(countries))]
gdp = gdp[["iso_2", "year", "gdp"]]

In [22]:
bi_tax = (
    its_eu28_ip[str_vars_red]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(tax, how="left", left_on="partner_iso_2", right_on="iso_2")
)
bi_tax.drop("iso_2", axis=1, inplace=True)

In [23]:
cfc_oecd = pd.read_excel("CFC_11072021_121508.xlsx", usecols="A,C", index_col=False)
cfc_oecd.rename(
    columns={cfc_oecd.columns[0]: "_iso_3", cfc_oecd.columns[1]: "cfc_since"},
    inplace=True,
)
cfc_oecd["iso_3"] = cfc_oecd._iso_3.apply(lambda i: re.search("\(([^)]+)", i).group(1))
cfc_oecd["iso_2"] = cfc_oecd.iso_3.map(cc_iso_3_2_map)
cfc_oecd = cfc_oecd[["iso_2", "cfc_since"]]
cfc_oecd.dropna(subset=["cfc_since"], inplace=True)

In [24]:
cfc_oecd_add = pd.DataFrame({"iso_2": ["CY"], "cfc_since": [2019]})
cfc_oecd = (
    pd.concat([cfc_oecd, cfc_oecd_add])
    .reset_index(drop=True)
    .sort_values(by=["iso_2"], ascending=True)
)
cfc_eu28 = cfc_oecd[cfc_oecd.iso_2.isin(eu28_iso_2)].reset_index(drop=True)
cfc_eu28.rename(columns={"iso_2": "geo_iso_2"}, inplace=True)

In [25]:
cfc_timespan = list(range(min(cfc_eu28["cfc_since"]), max(cfc_eu28["cfc_since"]) + 1))
cfc_eu28 = pd.concat(
    [
        cfc_eu28,
        pd.DataFrame(columns=cfc_timespan),
    ]
)
for i, i_content in cfc_eu28.iterrows():
    for j in cfc_timespan:
        if i_content.cfc_since <= j:
            cfc_eu28.iloc[i, cfc_eu28.columns.get_loc(j)] = 1
        else:
            cfc_eu28.iloc[i, cfc_eu28.columns.get_loc(j)] = 0
cfc_eu28.drop("cfc_since", axis=1, inplace=True)
cfc_eu28 = cfc_eu28[["geo_iso_2"] + its_timespan]

In [26]:
cfc_eu28_threshold = pd.DataFrame(
    cfc_eu28[its_timespan].values * tax_eu28[its_timespan].values,
    columns=its_timespan,
    index=tax_eu28.index,
)
cfc_eu28_threshold["geo_iso_2"] = tax_eu28.iso_2

In [27]:
cfc_eu28 = pd.melt(
    cfc_eu28,
    id_vars=["geo_iso_2"],
    value_vars=its_timespan,
    var_name="year",
    value_name="is_cfc",
)

In [28]:
FI_safelist = {
    "AR": 1996,
    "AU": 2007,
    "BE": 1995,
    "BR": 1997,
    "CA": 2007,
    "CY": 2013,
    "CZ": 1995,
    "DE": 1995,
    "DK": 1995,
    "EE": 1995,
    "EG": 1995,
    "GR": 1995,
    "ES": 1995,
    "FR": 1995,
    "HK": 2018,
    "HR": 1995,
    "HU": 1995,
    "ID": 1995,
    "IN": 1995,
    "IS": 1997,
    "IT": 1995,
    "JP": 1995,
    "KR": 1995,
    "MA": 1995,
    "MT": 2001,
    "MX": 1998,
    "NL": 1997,
    "NO": 1997,
    "NZ": 1995,
    "PH": 1995,
    "PL": 1995,
    "PT": 1995,
    "RO": 2000,
    "RU": 2002,
    "SE": 1997,
    "SI": 2004,
    "SK": 2000,
    "TH": 1986,
    "TR": 1995,
    "TW": 1995,
    "GB": 1995,
    "US": 1995,
    "ZA": 1995,
}
FI_safelist_ = list(FI_safelist.keys())
FI_blocklist = {"CH": 1995, "MY": 1995, "SG": 1995, "UY": 2014}
FI_blocklist_ = list(FI_blocklist.keys())

In [29]:
EE_safelist = eu28_iso_2 + [
    "CA",
    "IS",
    "IN",
    "JP",
    "MX",
    "NO",
    "SG",
    "KR",
    "CH",
    "TL",
    "US",
]
HU_safelist = eu28_iso_2 + ["IS", "LI", "NO"]
SE_safelist = ["AR", "CA", "CL", "MX", "NZ", "US", "VE"]
GB_safelist = [
    "AT",
    "AU",
    "BR",
    "BG",
    "CA",
    "CZ",
    "DE",
    "DK",
    "FI",
    "FR",
    "IS",
    "JP",
    "KR",
    "MX",
    "NG",
    "NO",
    "NZ",
    "PL",
    "RO",
    "SE",
    "SK",
    "US",
]

In [30]:
LV_blocklist = ["HK", "LI", "UY"]
LT_blocklist = LV_blocklist
SE_blocklist = ["LI"]

In [31]:
cfc_tax = bi_tax.copy()
for i, i_content in cfc_tax.iterrows():
    i_geo = i_content.geo_iso_2
    i_partner = i_content.partner_iso_2
    for j in its_timespan:
        cfc_safe = False
        cfc_block = False
        i_j_threshold = list(
            cfc_eu28_threshold[cfc_eu28_threshold.geo_iso_2 == i_geo][j]
        )[0]
        if i_geo in list(stricter_control_thresholds_fixed.keys()):
            i_j_threshold = stricter_control_thresholds_fixed[i_geo]
        elif i_geo in stricter_control_thresholds_share:
            i_j_threshold = control_threshold_share * i_j_threshold
        if i_content.geo_iso_2 == "FI":
            if i_content.partner_iso_2 in FI_safelist_:
                if j >= FI_safelist[i_partner]:
                    cfc_safe = True
            if i_content.partner_iso_2 in FI_blocklist_:
                if j >= FI_blocklist[i_partner]:
                    cfc_block = True
        if i_geo == "EE":
            if i_partner in EE_safelist:
                cfc_safe = True
        if i_geo == "HU":
            if i_partner in HU_safelist:
                cfc_safe = True
        if i_geo == "SE":
            if i_partner in SE_safelist:
                cfc_safe = True
            if i_partner in SE_blocklist:
                cfc_block = True
        if i_geo == "GB":
            if i_partner in GB_safelist:
                cfc_safe = True
        if i_geo == "LV":
            if i_partner in LV_blocklist:
                cfc_block = True
        if i_geo == "LT":
            if i_partner in LT_blocklist:
                cfc_block = True
        if i_j_threshold != 0:
            if (~cfc_safe & (i_content[j] <= i_j_threshold)) | cfc_block:
                cfc_tax.iloc[i, cfc_tax.columns.get_loc(j)] = i_j_threshold

In [32]:
bi_tax = pd.melt(
    bi_tax,
    id_vars=str_vars_red,
    value_vars=its_timespan,
    var_name="year",
    value_name="tax",
)

In [33]:
cfc_tax = pd.melt(
    cfc_tax,
    id_vars=str_vars_red,
    value_vars=its_timespan,
    var_name="year",
    value_name="cfc_tax",
).merge(
    cfc_eu28,
    how="left",
    on=["geo_iso_2", "year"],
)

In [34]:
for i, i_content in cfc_tax.iterrows():
    cfc_tax.loc[i, "all_tax"] = cfc_tax.loc[i, "cfc_tax"]
    if i_content.year != min(its_timespan):
        c_geo = cfc_tax.geo_iso_2 == i_content.geo_iso_2
        c_partner = cfc_tax.partner_iso_2 == i_content.partner_iso_2
        c_year_back = cfc_tax.year < i_content.year
        tax_year_back = list(
            cfc_tax[c_geo & c_partner & c_year_back].sort_values(
                by=["year"], ascending=False
            )["cfc_tax"]
        )
        if tax_year_back:
            tax_year_back = tax_year_back[0]
        if i_content.is_cfc == 0:
            if i_content.cfc_tax != tax_year_back:
                cfc_tax.loc[i, "cfc_tax"] = None

In [35]:
its_timespan_sql = "('" + "', '".join([str(i) for i in its_timespan]) + "')"
patents_inventor_countries = """
WITH selection AS (SELECT p.family_id,
       inv.country_code iso_2,
       SUBSTR(CAST(p.grant_date AS string), 1, 4) year
FROM   `patents-public-data.patents.publications` p,
       unnest(p.inventor_harmonized) inv
WHERE  p.application_kind = "A"
AND p.grant_date <> 0
AND SUBSTR(CAST(p.grant_date AS string), 1, 4) IN {0}
AND inv.country_code <> ""
GROUP BY p.family_id, inv.country_code, SUBSTR(CAST(p.grant_date AS string), 1, 4))
SELECT DISTINCT s1.iso_2 geo_iso_2, s2.iso_2 partner_iso_2, s1.year,
                COUNT(*) AS patent_collab_count
       FROM selection s1 join selection s2
       ON s1.family_id = s2.family_id
       WHERE s1.iso_2 <> s2.iso_2
       GROUP BY s2.iso_2, s1.iso_2, s1.year
""".format(
    its_timespan_sql
)
patents_inventor_countries = client.query(patents_inventor_countries).to_dataframe()
patents_inventor_countries = patents_inventor_countries[
    (
        patents_inventor_countries.geo_iso_2.isin(eu28_iso_2)
        | patents_inventor_countries.partner_iso_2.isin(eu28_iso_2)
    )
]
patents_inventor_countries = patents_inventor_countries[
    (
        patents_inventor_countries.geo_iso_2.isin(countries)
        & patents_inventor_countries.partner_iso_2.isin(countries)
    )
].reset_index(drop=True)
patents_inventor_countries["year"] = pd.to_numeric(patents_inventor_countries["year"])

In [36]:
patents_inventor_countries_ = patents_inventor_countries.rename(
    columns={"geo_iso_2": "partner_iso_2", "partner_iso_2": "geo_iso_2"}
)
patents_inventor_countries = patents_inventor_countries.append(
    patents_inventor_countries_
).reset_index(drop=True)

In [37]:
patents_inventor_countries_[["geo_iso_2", "partner_iso_2"]] = np.sort(
    patents_inventor_countries_[["geo_iso_2", "partner_iso_2"]], axis=1
)
patents_inventor_countries_sum = (
    patents_inventor_countries_.groupby(str_vars_red)["patent_collab_count"]
    .agg("sum")
    .reset_index()
)

In [38]:
patents_inventor_countries_sum_within = patents_inventor_countries_sum[
    (patents_inventor_countries_sum.partner_iso_2.isin(eu28_iso_2))
    & (patents_inventor_countries_sum.geo_iso_2.isin(eu28_iso_2))
].reset_index(drop=True)
patents_inventor_countries_sum_within[
    ["geo_iso_2", "partner_iso_2", "patent_collab_count"]
].to_csv("patents_inventor_countries_sum_within.csv", index=False)

In [39]:
patents_inventor_countries_sum_outside = patents_inventor_countries_sum[
    ((~patents_inventor_countries_sum.geo_iso_2.isin(eu28_iso_2)) | (~patents_inventor_countries_sum.partner_iso_2.isin(eu28_iso_2)))
].reset_index(drop=True)
patents_inventor_countries_sum_outside[
    ["geo_iso_2", "partner_iso_2", "patent_collab_count"]
].to_csv("patents_inventor_countries_sum_outside.csv", index=False)

In [40]:
wipo_gii = pd.DataFrame()
for i in list(range(2013, 2020)):
    if i == 2019:
        gii_i = pd.read_csv("wipo-gii/{0}.csv".format(str(i)), index_col=False)[2:4]
    else:
        gii_i = pd.read_csv("wipo-gii/{0}.csv".format(str(i)), index_col=False)[3:5]
    gii_i.rename(columns={" ": "index"}, inplace=True)
    gii_i_cols_ = [i for i in gii_i.columns if ".1" in i][1:]
    gii_i_cols = [i.replace(".1", "") for i in gii_i_cols_]
    gii_i = gii_i[["index"] + gii_i_cols_].reset_index(drop=True)
    gii_i.rename(columns=dict(zip(gii_i_cols_, gii_i_cols)), inplace=True)
    gii_i.iloc[0, 0] = "innovation_input"
    gii_i.iloc[1, 0] = "innovation_output"
    gii_i = gii_i.set_index("index").T.reset_index().rename_axis(None, axis=1)
    gii_i.rename(columns={"index": "country"}, inplace=True)
    gii_i["year"] = i
    wipo_gii = wipo_gii.append(gii_i).reset_index(drop=True)

In [41]:
for i in [
    " (China)",
    " (the)",
    ", Plurinational St.",
    ", China",
    ", Islamic Rep.",
    ", United Rep.",
    "United Republic of ",
    " (Plurinational State of)",
    " of America",
    " (Islamic Republic of)",
    ", Bolivarian Rep.",
]:
    wipo_gii["country"] = wipo_gii["country"].str.replace(i, "", regex=False)
for k, v in {"Republic of Korea": "South Korea"}.items():
    wipo_gii["country"] = wipo_gii["country"].str.replace(k, v, regex=False)
wipo_gii["iso_2"] = wipo_gii.country.map(cc_country_iso_2_map)
wipo_gii = wipo_gii[wipo_gii.iso_2.isin(countries)][
    ["iso_2", "year", "innovation_input", "innovation_output"]
].reset_index(drop=True)

In [42]:
wipo_innovation_2010 = pd.read_html(
    requests.get(
        "http://stats.areppim.com/listes/list_innovation_2009_2010.htm"
    ).content
)[0].loc[:, 0:1]
wipo_innovation_2010.columns = ["country", "score"]
wipo_innovation_2010["country"] = wipo_innovation_2010["country"].replace(
    {
        "CÃ´te d'Ivoire": "Côte d'Ivoire",
        "Dominican": "Dominican Republic",
        "Egypt, Arab Rep.": "Egypt",
        "Gambia, The": "Gambia",
        "Hong Kong, China": "Hong Kong",
        "Iran": "Iran, Islamic Republic of",
        "Korea, Rep.": "South Korea",
        "Kyrgyz Republic": "Kyrgyzstan",
        "Macedonia, FYR": "Macedonia, the former Yugoslav Republic of",
        "Slovak Republic": "Slovakia",
        "Tanzania": "Tanzania, United Republic of",
        "Venezuela, RB": "Venezuela",
    }
)
wipo_innovation_2010 = wipo_innovation_2010[136:267].merge(
    wipo_innovation_2010[270:402],
    how="left",
    on="country",
)
wipo_innovation_2010[["score_x", "score_y"]] = wipo_innovation_2010[
    ["score_x", "score_y"]
].astype(float)
wipo_innovation_2010["innovation_input"] = wipo_innovation_2010["score_x"] * 100/7
wipo_innovation_2010["innovation_output"] = wipo_innovation_2010["score_y"] * 100/7
wipo_innovation_2010["year"] = 2010

In [43]:
wipo_innovation_2011 = pd.read_html(
    requests.get("http://stats.areppim.com/listes/list_innovation_2011.htm").content
)[1].loc[:, 0:1]
wipo_innovation_2011.columns = ["country", "score"]
wipo_innovation_2011["country"] = wipo_innovation_2011["country"].replace(
    {
        "Hong Kong (SAR), China": "Hong Kong",
        "Iran": "Iran, Islamic Republic of",
        "Korea, Rep.": "South Korea",
        "Macedonia": "Macedonia, the former Yugoslav Republic of",
        "Moldova, Rep": "Moldova, Republic of",
        "Moldova, Rep.": "Moldova, Republic of",
        "Slovak Republic": "Slovakia",
        "Syrian Arab Rep.": "Syrian Arab Republic",
        "Tanzania": "Tanzania, United Republic of",
        "United State": "United States",
    }
)
wipo_innovation_2011 = wipo_innovation_2011[2:126].merge(
    wipo_innovation_2011[129:253],
    how="left",
    on="country",
)
wipo_innovation_2011[["innovation_input", "innovation_output"]] = wipo_innovation_2011[
    ["score_x", "score_y"]
].astype(float)
wipo_innovation_2011["year"] = 2011

In [44]:
wipo_innovation_2012 = pd.read_html(
    requests.get("http://stats.areppim.com/listes/list_innovation_2012.htm").content
)[0].loc[:, 0:1]
wipo_innovation_2012.columns = ["country", "score"]
wipo_innovation_2012["country"] = wipo_innovation_2012["country"].replace(
    {
        "Bolivia, Plurinational St.": "Bolivia",
        "Hong Kong (China)": "Hong Kong",
        "Iran, Islamic Rep.": "Iran, Islamic Republic of",
        "Korea, Rep.": "South Korea",
        "Lao PDR": "Lao People's Democratic Republic",
        "Macedonia, FYR": "Macedonia, the former Yugoslav Republic of",
        "Moldova, Rep.": "Moldova, Republic of",
        "Syrian Arab Rep.": "Syrian Arab Republic",
        "Tanzania, United Rep.": "Tanzania, United Republic of",
        "Trinidad And Tobago": "Trinidad and Tobago",
        "Venezuela, Bolivarian Rep.": "Venezuela",
    }
)
wipo_innovation_2012 = wipo_innovation_2012[145:285].merge(
    wipo_innovation_2012[288:429],
    how="left",
    on="country",
)
wipo_innovation_2012[["innovation_input", "innovation_output"]] = wipo_innovation_2012[
    ["score_x", "score_y"]
].astype(float)
wipo_innovation_2012["year"] = 2012

In [45]:
wipo_gii_add = (
    wipo_innovation_2010
    .append(wipo_innovation_2011)
    .append(wipo_innovation_2012)[
        ["country", "year", "innovation_input", "innovation_output"]
    ]
    .reset_index(drop=True)
)

In [46]:
wipo_gii_add["iso_2"] = wipo_gii_add.country.map(cc_country_iso_2_map)
wipo_gii_add = wipo_gii_add[
    ["iso_2", "year", "innovation_input", "innovation_output"]
]

In [47]:
wipo_gii = wipo_gii.append(wipo_gii_add)
wipo_gii = wipo_gii[wipo_gii.iso_2.isin(countries)].reset_index(
    drop=True
)

In [48]:
wipo_gii_tax = wipo_gii.copy()
wipo_gii_tax = wipo_gii_tax.merge(tax_long, on=["iso_2", "year"])
wipo_gii_tax_2019 = wipo_gii_tax[(wipo_gii_tax.year == 2019)].reset_index(drop=True)
wipo_gii_tax_2019.to_csv("wipo_gii_tax_2019.csv", index=False)

In [49]:
wipo_gii_input = wipo_gii[["iso_2", "year", "innovation_input"]]
wipo_gii_output = wipo_gii[["iso_2", "year", "innovation_output"]]

In [50]:
wipo_gii_input = pd.pivot_table(wipo_gii_input,
              columns="year",
              values="innovation_input",
              index="iso_2",)
wipo_gii_output = pd.pivot_table(wipo_gii_output,
              columns="year",
              values="innovation_output",
              index="iso_2",)

In [51]:
wipo_gii_input = wipo_gii_input.T.fillna(
    wipo_gii_input.mean(axis=1)
).T.rename_axis(None, axis=1).reset_index()
wipo_gii_output = wipo_gii_output.T.fillna(
    wipo_gii_output.mean(axis=1)
).T.rename_axis(None, axis=1).reset_index()

In [52]:
wipo_gii_input = pd.melt(
    wipo_gii_input,
    id_vars=["iso_2"],
    value_vars=its_timespan,
    var_name="year",
    value_name="innovation_input",
)
wipo_gii_output = pd.melt(
    wipo_gii_output,
    id_vars=["iso_2"],
    value_vars=its_timespan,
    var_name="year",
    value_name="innovation_output",
)
wipo_gii = wipo_gii_input.merge(wipo_gii_output, on=["iso_2","year"])

In [67]:
cfc_tax

,partner_iso_2,geo_iso_2,year,cfc_tax,is_cfc,all_tax
0,AR,AT,2010,35.0,0,35.0
1,AR,BE,2010,35.0,0,35.0
2,AR,BG,2010,35.0,0,35.0
3,AR,CY,2010,35.0,0,35.0
4,AR,CZ,2010,35.0,0,35.0
...,...,...,...,...,...,...
13005,ZA,PL,2019,28.0,1,28.0
13006,ZA,RO,2019,28.0,1,28.0
13007,ZA,SE,2019,28.0,1,28.0
13008,ZA,SI,2019,28.0,1,28.0


In [68]:
innovation_cfc_tax = (
    its_eu28_ip.merge(
        bi_tax,
        how="left",
        on=str_vars_red + ["year"],
    )
    .merge(
        cfc_tax,
        how="left",
        on=str_vars_red + ["year"],
    )
    .merge(gdp, how="left", left_on=["geo_iso_2", "year"], right_on=["iso_2", "year"], suffixes=('_x', '_y'))
    .rename(columns={"gdp": "geo_gdp"})
    .merge(
        gdp, how="left", left_on=["partner_iso_2", "year"], right_on=["iso_2", "year"], suffixes=('_xx', '_yy')
    )
    .rename(columns={"gdp": "partner_gdp"})
    .merge(
        wipo_gii,
        how="left",
        left_on=["geo_iso_2", "year"],
        right_on=["iso_2", "year"],
    )
    .rename(
        columns={
            "innovation_input": "geo_innovation_input",
            "innovation_output": "geo_innovation_output",
        }
    )
    .merge(
        wipo_gii,
        how="left",
        left_on=["partner_iso_2", "year"],
        right_on=["iso_2", "year"],
    )
    .rename(
        columns={
            "innovation_input": "partner_innovation_input",
            "innovation_output": "partner_innovation_output",
        }
    )
    .merge(patents_inventor_countries, how="left", on=str_vars_red+["year"])
)

In [69]:
innovation_cfc_tax = innovation_cfc_tax[
    [
        "iso_2_pair",
        "geo_iso_2",
        "partner_iso_2",
        "year",
        "ip_export",
        "is_cfc",
        "cfc_tax",
        "all_tax",
        "geo_gdp",
        "partner_gdp",
        "patent_collab_count",
        "geo_innovation_input",
        "partner_innovation_input",
        "geo_innovation_output",
        "partner_innovation_output",
    ]
]

In [70]:
innovation_cfc_tax["patent_collab_count"] = innovation_cfc_tax["patent_collab_count"].fillna(0)

In [71]:
distinct_iso_2_pair = len(innovation_cfc_tax.iso_2_pair.unique())
distinct_iso_2 = len(countries)

In [72]:
transfer_count = innovation_cfc_tax.ip_export.count()
transfer_described = innovation_cfc_tax.ip_export.describe()
transfer_within_described = innovation_cfc_tax[
    innovation_cfc_tax.partner_iso_2.isin(eu28_iso_2)
].ip_export.describe()
transfer_outside_described = innovation_cfc_tax[
    ~innovation_cfc_tax.partner_iso_2.isin(eu28_iso_2)
].ip_export.describe()

In [73]:
gii_corr_2019 = np.corrcoef(wipo_gii_tax_2019.tax, wipo_gii_tax_2019.innovation_input)

In [74]:
collab_sum = sum(patents_inventor_countries_sum.patent_collab_count)
collab_within_described = patents_inventor_countries_sum_within.describe()
collab_outside_described = patents_inventor_countries_sum_outside.describe()

In [77]:
innovation_cfc_tax["ip_export"] += 0.4
innovation_cfc_tax["patent_collab_count"] += 0.4

In [78]:
innovation_cfc_tax.to_csv("innovation_cfc_tax.csv", index=False)